In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, pipeline
# import lmap
from datasets import load_dataset
from evaluate import load
from src.data_collator import linearise_input, convert_to_features
import wandb
import os
import numpy as np
import torch.nn.functional as F
import torch
from tqdm import tqdm
import json
import yaml
import argparse
from transformers.trainer_callback import EarlyStoppingCallback
import random

/home/james/CodingProjects/Local_level_model_explanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-10 12:01:55.012498: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 12:01:55.541260: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-12-10 12:01:55.541359: W tensorflow/compiler/xla/stream_executor/platform/def

In [2]:

model = AutoModelForSeq2SeqLM.from_pretrained('t5-base', return_dict=True)
tokenizer = AutoTokenizer.from_pretrained('t5-base')

dataset = load_dataset("james-burton/textual-explanations")

/home/james/CodingProjects/Local_level_model_explanations/env/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Using custom data configuration james-burton--textual-explanations-19ff8605823ae74a
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--t

In [3]:
dataset = dataset.map(
        lambda x: linearise_input(x, 'essel', 15),
        load_from_cache_file=False)

100%|██████████| 47/47 [00:00<00:00, 2890.25ex/s]


In [4]:
dataset['train'][0]['input']

"| predicted class | C1 64.66% | other classes | C2 35.34% |     features | F5 && F8 && F1 && F11 && F7 && F6 && F16 && F15 && F13 && F3 && F14 && F9 && F10 && F2 && F12 | postive features | F5 && F8 && F1 && F7 && F16 && F15 && F3 && F14 && F9 && F10  | negative features | F11 && F6 && F13 && F2 && F12  |     negligible features | None |\n <br> <br> Using the above information, answer the following             in detail: <br> <br> 1. For this test instance, provide information on the predicted label along with the confidence level of the model's decision.\n2. Summarize the top features influencing the model's decision along with the respective directions of influence on the prediction?\n3. Summarize the direction of influence of the features (F6, F16 and F15) with moderate impact on the prediction made for this test case."

Things I need to generate:
* predicted_class
* classes_dict
* feature_nums
* sign
* values
*

In [5]:
dataset['train'][0]['classes_dict']

"{'C2': '35.34%', 'C1': '64.66%'}"

In [21]:
def create_classes_dict():
    sign_dict = {1: 'positive', -1: 'negative', 0: 'negligible'}
    x = dict()
    classes = ['C1', 'C2']
    random.shuffle(['C1', 'C2'])
    pct = random.randint(500,9500)/100
    other_pct = round(100 - pct, 2)
    class_dict  = {classes[0]: format(pct, '.2f'), classes[1]: format(other_pct, '.2f')}
    x['predicted_class'] = max(class_dict, key=class_dict.get)
    x['classes_dict'] = str({f'{k}': f'{v}%' for k,v in class_dict.items()})
    x['feature_nums'] = [f'F{i}' for i in random.sample(range(1, 80), 15)]
    values = sorted([random.randint(-50,50)/100 for i in range(15)], key=abs, reverse=True)
    x['sign'] = [sign_dict[np.sign(i)] for i in values]
    x['values'] =  x['values'] = [format(v, '.2f') for v in values]
    return x

def question_generator(dict):
    # 'What is the value of FA?'
    choice = random.randint(0,len(dict['feature_nums'])-1)
    feat = dict['feature_nums'][choice]
    q1 = f'What is the value of {feat}?'
    val = dict['values'][choice]
    a1 = val
    
    # What is FA - FB?
    choice1 = random.randint(0,len(dict['feature_nums'])-1)
    feat1 = dict['feature_nums'][choice1]
    val1 = dict['values'][choice1]
    choice2 = random.randint(0,len(dict['feature_nums'])-1)
    feat2 = dict['feature_nums'][choice2]
    val2 = dict['values'][choice2]
    q2 = f'What is the difference between {feat1} and {feat2}?'
    a2 = format(float(val1) - float(val2), '.2f')
    
    # What is the xth most important feature?
    choice = random.randint(0,len(dict['feature_nums'])-1)
    feat = dict['feature_nums'][choice]
    q3 = f'What is the {choice+1}th most important feature?'
    a3 = feat
    
    # Top x postive features: 
    x = random.randint(1,5)
    top_x_pos_fts = [ft for ft, sign in zip(dict['feature_nums'],dict['sign']) if sign == 'positive'][:x]
    q4 = f'What are the top {x} positive features?'
    a4 = ', '.join(top_x_pos_fts)
    
    # Top x negative features: 
    x = random.randint(1,5)
    top_x_neg_fts = [ft for ft, sign in zip(dict['feature_nums'],dict['sign']) if sign == 'negative'][:x]
    q4 = f'What are the top {x} negative features?'
    a4 = ', '.join(top_x_neg_fts)
    
    dict['questions'] = [q1, q2, q3, q4]
    dict['answers'] = [a1, a2, a3, a4]
    
    return dict

In [13]:
example_dict = create_classes_dict()
linearise_input(example_dict, 'ord_first', 15, data_only=True)

{'predicted_class': 'C2',
 'classes_dict': "{'C1': '11.85%', 'C2': '88.15%'}",
 'feature_nums': ['F65',
  'F68',
  'F16',
  'F51',
  'F12',
  'F8',
  'F61',
  'F45',
  'F30',
  'F13',
  'F40',
  'F63',
  'F56',
  'F21',
  'F77'],
 'sign': ['positive',
  'negative',
  'negative',
  'negative',
  'negative',
  'negative',
  'positive',
  'positive',
  'positive',
  'positive',
  'negative',
  'positive',
  'positive',
  'negative',
  'negative'],
 'values': ['0.50',
  '-0.49',
  '-0.49',
  '-0.47',
  '-0.47',
  '-0.45',
  '0.35',
  '0.35',
  '0.34',
  '0.28',
  '-0.23',
  '0.22',
  '0.20',
  '-0.11',
  '-0.01'],
 'input': '| predicted class | C2 88.15% | other classes | C1 11.85% | 1st | F65 positive 0.50 | 2nd | F68 negative -0.49 | 3rd | F16 negative -0.49 | 4th | F51 negative -0.47 | 5th | F12 negative -0.47 | 6th | F8 negative -0.45 | 7th | F61 positive 0.35 | 8th | F45 positive 0.35 | 9th | F30 positive 0.34 | 10th | F13 positive 0.28 | 11th | F40 negative -0.23 | 12th | F63 positiv

* Feature importance score of F#
* What is F#1 - F#2?
* xth most important feature?
* Top 2,3,4,5 positive features?
* Top 2,3,4,5 negative features?

In [20]:
[ft for ft, sign in zip(example_dict['feature_nums'],example_dict['sign']) if sign == 'positive']

['F65', 'F61', 'F45', 'F30', 'F13', 'F63', 'F56']

In [16]:
random.randint(0,len(example_dict['feature_nums'])-1)

0

In [ ]:
def question_generator(dict):
    # 'What is the value of FA?'
    choice = random.randint(0,len(example_dict['feature_nums'])-1)
    feat = example_dict['feature_nums'][choice]
    q1 = f'What is the value of {feat}?'
    val = example_dict['values'][choice]
    a1 = val
    
    # What is FA - FB?
    choice1 = random.randint(0,len(example_dict['feature_nums'])-1)
    feat1 = example_dict['feature_nums'][choice1]
    val1 = example_dict['values'][choice1]
    choice2 = random.randint(0,len(example_dict['feature_nums'])-1)
    feat2 = example_dict['feature_nums'][choice2]
    val2 = example_dict['values'][choice2]
    q2 = f'What is the difference between {feat1} and {feat2}?'
    a2 = format(float(val1) - float(val2), '.2f')
    
    # What is the xth most important feature?
    choice = random.randint(0,len(example_dict['feature_nums'])-1)
    feat = example_dict['feature_nums'][choice]
    q3 = f'What is the {choice+1}th most important feature?'
    a3 = feat
    
    # Top x postive features: 
    x = random.randint(1,5)
    top_x_pos_fts = [ft for ft, sign in zip(example_dict['feature_nums'],example_dict['sign']) if sign == 'positive'][:x]
    q4 = f'What are the top {x} positive features?'
    a4 = ', '.join(top_x_pos_fts)
    
    # Top x negative features: 
    x = random.randint(1,5)
    top_x_neg_fts = [ft for ft, sign in zip(example_dict['feature_nums'],example_dict['sign']) if sign == 'negative'][:x]
    q4 = f'What are the top {x} negative features?'
    a4 = ', '.join(top_x_neg_fts)
    
    dict['quesitons'] = [q1, q2, q3, q4]
    dict['answers'] = [a1, a2, a3, a4]
    
    return dict